In [48]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import cycle

sns.set_theme(style="white", palette=None)
color_pal = plt.rcParams["axes.prop_cycle"].by_key()["color"]
color_cycle = cycle(plt.rcParams["axes.prop_cycle"].by_key()["color"])

In [50]:
import sys
sys.path.append('../src')
from data.audio_dataset import AudioSpectrogramDataset
from models.PGAN_model.PGenerator import PGenerator
from models.PGAN_model.PDiscriminator import PDiscriminator
from models.WGAN_model.config import *

import torch
from torch.utils.data import DataLoader

In [30]:
# # hyperparameters
# z_dim = 100
batch_size = 32
num_epochs = 50
latent_dim = 100
# learning_rate = 0.0002
# beta1 = 0.5

# base_directory = "../data/raw/NSynth/audio"
base_directory = "../data/raw/Single-bass"

dataset = AudioSpectrogramDataset(base_directory=base_directory, spectro_type='mel')
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [14]:
first_batch = next(iter(dataloader))
print(first_batch.size())

torch.Size([1, 1, 1024, 160])


C:\Users\micha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\librosa\feature\spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)


In [51]:
temp = torch.rand((64, 1, 8, 5))
# temp = torch.unsqueeze(temp, dim=0)
print(temp.shape)

torch.Size([64, 1, 8, 5])


In [57]:
import torch.nn.functional as F
goal_size = [64, 1, 32, 5]
goal = (32, 5)

In [8]:
goal_size[-2:]

[128, 80]

In [58]:
low_res_real = F.adaptive_avg_pool2d(temp, output_size=goal)
low_res_real2 = F.interpolate(temp, size=goal_size[-2:], mode='nearest')

In [59]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    n_devices = torch.cuda.device_count()
else:
    device = torch.device("cpu")
    n_devices = 1    
device

device(type='cuda', index=0)

In [60]:
print(f'low_res_real: {low_res_real.shape}')
print(f'low_res_real2: {low_res_real2.shape}')

low_res_real: torch.Size([64, 1, 32, 5])
low_res_real2: torch.Size([64, 1, 32, 5])


In [73]:
temp = torch.rand((64, 1, 128, 20))
# temp = torch.unsqueeze(temp, dim=0)
print(temp.shape)

torch.Size([64, 1, 128, 20])


In [75]:
discriminator = PDiscriminator(
    init_resolution_size=init_resolution_size,
    mini_batch_normalization=False).to(device)
low_res_real = temp.to(device)
discriminator.add_next_block(new_depth=256)
discriminator.add_next_block(new_depth=128)
# discriminator.add_next_block(new_depth=64)
# discriminator.add_next_block(new_depth=32)
# discriminator.add_next_block(new_depth=16)
# discriminator.set_alpha(0.2)
discriminator.set_alpha(1.0)
# print(first_batch_low_resolution.device)
dummy_output = discriminator(low_res_real)
print(dummy_output.shape)
# print(dummy_output)

input: torch.Size([64, 1, 128, 20])
before final block: torch.Size([64, 256, 32, 5])
after final block: torch.Size([64, 1, 1, 1])
torch.Size([64, 1, 1, 1])


In [45]:
p_latent_dim = 256
dummy_latent_vector = torch.randn((64, p_latent_dim, 1, 1)).to(device)
generator = PGenerator(
    latent_dim=p_latent_dim,
    init_resolution_size=init_resolution_size,
    output_depth=output_dim,
    LReLU_negative_slope=negative_slope,
    normalization=normalization
).to(device)
generator.add_next_block(new_depth=256)
generator.add_next_block(new_depth=128)
generator.add_next_block(new_depth=64)
# generator.add_next_block(new_depth=32)
# generator.add_next_block(new_depth=16)
generator.set_alpha(1.0)
# generator.set_alpha(0.2)
dummy_output = generator(dummy_latent_vector)
print(dummy_output.shape)

input_size: torch.Size([64, 256, 1, 1])
after base block size: torch.Size([64, 256, 32, 5])
shape before upsampling: torch.Size([64, 256, 32, 5])
input of upsampling: torch.Size([64, 256, 32, 5])
shape after upsampling: torch.Size([64, 256, 64, 10])
after block: torch.Size([64, 256, 64, 10])
shape before upsampling: torch.Size([64, 256, 64, 10])
input of upsampling: torch.Size([64, 256, 64, 10])
shape after upsampling: torch.Size([64, 256, 128, 20])
after block: torch.Size([64, 128, 128, 20])
shape before upsampling: torch.Size([64, 128, 128, 20])
input of upsampling: torch.Size([64, 128, 128, 20])
shape after upsampling: torch.Size([64, 128, 256, 40])
after block: torch.Size([64, 64, 256, 40])
torch.Size([64, 1, 256, 40])


In [36]:
print(generator)

PGenerator2(
  (blocks): ModuleList(
    (0): ModuleList(
      (0): Sequential(
        (0): Upsample(scale_factor=2.0, mode=nearest)
        (1): ConvTranspose2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (2): LeakyReLU(negative_slope=0.2)
        (3): ConvTranspose2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): LeakyReLU(negative_slope=0.2)
      )
      (1): Sequential(
        (0): Conv2d(256, 1, kernel_size=(1, 1), stride=(1, 1))
        (1): Upsample(scale_factor=2.0, mode=nearest)
      )
    )
  )
  (normalizationLayer): NormalizationLayer()
  (l1): Sequential(
    (0): Linear(in_features=100, out_features=10240, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
  )
  (base_block): ModuleList(
    (0): Sequential(
      (0): ConvTranspose2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.2)
    )
    (1): Sequential(
      (0): Conv2d(256, 1, kernel_size=(1, 1), stride=(1,

In [47]:
def create_noise(batch_size):
    noise = torch.randn(batch_size, 256, 1, 1).to(device)
    return noise

noise = create_noise(64)
print(noise.shape)
print(noise)


torch.Size([64, 256, 1, 1])
tensor([[[[ 0.7765]],

         [[ 1.1099]],

         [[-0.6654]],

         ...,

         [[-0.9404]],

         [[ 0.5894]],

         [[ 0.4751]]],


        [[[ 0.1632]],

         [[-1.3964]],

         [[-0.6469]],

         ...,

         [[-0.1766]],

         [[-1.4044]],

         [[ 1.4762]]],


        [[[ 1.0196]],

         [[ 0.5363]],

         [[ 1.5771]],

         ...,

         [[-1.1787]],

         [[-0.2667]],

         [[-0.3476]]],


        ...,


        [[[ 0.0200]],

         [[ 0.2189]],

         [[ 0.2410]],

         ...,

         [[-0.1960]],

         [[ 0.3864]],

         [[ 1.6617]]],


        [[[-1.5526]],

         [[-0.8472]],

         [[ 0.8952]],

         ...,

         [[-1.3719]],

         [[ 0.7228]],

         [[ 0.7435]]],


        [[[-0.9797]],

         [[-0.0460]],

         [[-1.1323]],

         ...,

         [[ 0.7799]],

         [[ 0.9960]],

         [[ 0.6180]]]], device='cuda:0')
